# Regression test case selection unisng quantum algorithms
The aim of this project is to exploit the capabilities of quantum algorithms and quantum computers to optimize the process of test case selection for regression testing purposes. In particular, representing every test case as a single qubit, that could be in a superposition of both the states 0 and 1 (meaning respectively "not selected" and "selected"), quantum algorithms are able to evaluate the sub-sets of the original test suite that, preserving the coverage level of the original test suite, optimize the better the cost consumption and fault-revealing properties. 

In [1]:
#this cell contains all the imports needed by the pipeline
import pandas

In [ ]:
#this cell contains all variable definitions that will be useful throughout the entire project
sir_programs=["flex","grep","gzip","sed"]

## The pipeline dataset
To experiment the performance of the proposed solution by this work and to compare its results to those of state-of-the-art solutions, 4 public programs have been downloaded from the SIR website. SIR is a repository of software-related artifacts meant to support rigorous controlled experimentation with program analysis and software testing techniques, and education in controlled experimentation. 

### Chosen SIR Programs
The programs that will be used for experimentation have all been written in C and are:
- flex (a program that generates a lexical analysis program, based on regular expressions and C statement contained in one or more input files);
- grep (a useful program to search form matching patterns in a file);
- gzip (a program that substitute a file, generally text files or web pages, with their compressed version)
- sed (a powerful program for stream text editing).

### Needed information
The information needed by the quantum algorithm to work on every one of the 4 programs are:
- a fault matrix: it indicates whether a precise test case already found, during previous execution, a bug in the source code or not;
- execution cost: it indicates the summation of the execution costs of all the test cases of the suite;
- statement coverage: it indicates statement coverage information for every test case. 

All this information have been gathered through previous experimentation on the four programs mentioned above and written in files organized in the SIR_Programs folder. So, the first goal of the project will be gathering data from these files for computational purposes.

In [ ]:
#let's start making a function to read the fault matrices
#IMPORTANT: all the fault-matrix files must be renamed as "fault-matrix".txt and must be written using the same standard used by the files of this project
def get_fault_matrix(program_name:str):
    program_file = open("/SIR_Programs/"+program_name+"/fault-matrix.txt")
    pass